In [1]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.data_manager import SHREDDataManager
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED
from models.shred_models import evaluate

In [ ]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

# Initialize SHREDDataManager
manager = SHREDDataManager(
    lags = 20,
    train_size = train_size,
    val_size = val_size,
    test_size = test_size,
    compression = 200,
    scaling = True,
    time=np.arange(0, 1600),
    method="reconstruct"
)


# Add data to manager (no sensors)
input_folder = "data14fields_npz"
for filename in os.listdir(input_folder):
    print('filename',filename)
    if filename.endswith(".npz") and filename != 'ni_3D.npz':
        file_path = os.path.join(input_folder, filename)
        data = np.load(file_path)
        data = data[data.files[0]]
        ntimesteps = data.shape[0]
        data = data[:-int(ntimesteps*test_size)]
        manager.add(
            data=data,
            id = str(os.path.splitext(filename)[0])
            # random_sensors=3,
        )
        print(f"Processed: {filename}")

# Add data to manager (with sensors)
filename = 'ni_3D.npz'
file_path = os.path.join(input_folder, filename)
ni_3D = np.load(file_path)
ni_3D = ni_3D[ni_3D.files[0]]

ntimesteps = ni_3D.shape[0]
ni_3D_reconstruction_train_for_prediction = ni_3D[:-int(ntimesteps*test_size)]
print(ni_3D_reconstruction_train_for_prediction.shape)


manager.add(
    data=ni_3D_reconstruction_train_for_prediction,
    stationary_sensors=[(29, 201), (76, 26), (11, 247)],
    id = str(os.path.splitext(filename)[0])
)


filename Jez_3D.npz
Processed: Jez_3D.npz
filename ne_3D.npz
Processed: ne_3D.npz
filename Jey_3D.npz
Processed: Jey_3D.npz
filename EfZ_3D.npz
Processed: EfZ_3D.npz
filename Vdiz_3D.npz
Processed: Vdiz_3D.npz
filename Tex_3D.npz
Processed: Tex_3D.npz
filename EfX_3D.npz
Processed: EfX_3D.npz
filename Jex_3D.npz
Processed: Jex_3D.npz
filename ni_3D.npz
filename Vdiy_3D.npz
Processed: Vdiy_3D.npz
filename phi_3D.npz
Processed: phi_3D.npz
filename Ti_3D.npz
Processed: Ti_3D.npz
filename Vdix_3D.npz
Processed: Vdix_3D.npz
filename Tez_3D.npz
Processed: Tez_3D.npz
(1600, 257, 256)


In [6]:
manager.input_summary

,field id,sensor id,sensor type,location/trajectory
0,ni_3D,ni_3D-0,stationary (user selected),"(29, 201)"
1,ni_3D,ni_3D-1,stationary (user selected),"(76, 26)"
2,ni_3D,ni_3D-2,stationary (user selected),"(11, 247)"


In [7]:
manager.sensor_measurements

,time,ni_3D-0,ni_3D-1,ni_3D-2
0,0,5.263100e+16,5.625600e+16,5.009700e+16
1,1,5.315000e+16,5.802500e+16,5.040600e+16
2,2,5.405600e+16,5.980300e+16,4.995000e+16
3,3,5.479300e+16,5.928200e+16,4.755900e+16
4,4,5.432100e+16,5.721300e+16,4.576400e+16
...,...,...,...,...
1595,1595,8.186700e+16,1.517500e+17,5.189900e+16
1596,1596,7.662800e+16,1.362600e+17,5.102800e+16
1597,1597,6.785100e+16,1.251700e+17,6.047400e+16
1598,1598,6.572600e+16,1.173200e+17,5.825500e+16


In [19]:
# Get train/val/test datasets
train_set, val_set, test_set = manager.preprocess()

# Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set.reconstructor_dataset.X.shape)
print('train Y:', train_set.reconstructor_dataset.Y.shape)
print('val X:', val_set.reconstructor_dataset.X.shape)
print('val Y:', val_set.reconstructor_dataset.Y.shape)
print('test X:', test_set.reconstructor_dataset.X.shape)
print('test Y:', test_set.reconstructor_dataset.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([960, 21, 3])
train Y: torch.Size([960, 2800])
val X: torch.Size([320, 21, 3])
val Y: torch.Size([320, 2800])
test X: torch.Size([320, 21, 3])
test Y: torch.Size([320, 2800])


In [20]:
num_epochs = 2000
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred.fit(train_set, val_set, num_epochs=num_epochs, patience=200, verbose=True)


Fitting Reconstructor...


Epoch 1/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

Epoch 2/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

Epoch 3/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

Epoch 4/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

...

Epoch 1540/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

Epoch 1541/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

Epoch 1542/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

Epoch 1543/2000:   0%|          | 0/15 [00:00<?, ?batch/s]

Early stopping triggered: patience threshold reached.


In [10]:
evaluate(shred = shred, test_dataset=test_set, data_manager=manager, uncompress=True, unscale = True)

,Jez_3D,ne_3D,Jey_3D,EfZ_3D,Vdiz_3D,Tex_3D,EfX_3D,Jex_3D,Vdiy_3D,phi_3D,Ti_3D,Vdix_3D,Tez_3D,ni_3D
reconstruction,0.052352,0.056261,0.37199,0.542862,0.468714,0.054374,0.07045,0.335182,0.056251,0.047878,0.074108,0.060818,0.049361,0.082303


In [ ]:
# Initialize SHREDDataManager
manager2 = SHREDDataManager(
    lags = 20,
    train_size = train_size,
    val_size = val_size,
    test_size = test_size,
    compression = 200,
    scaling = True,
    time=np.arange(0, 2000),
    method="all"
)


# Add data to manager (no sensors)
input_folder = "data14fields_npz"
for filename in os.listdir(input_folder):
    print('filename',filename)
    if filename.endswith(".npz") and filename != 'ni_3D.npz':
        file_path = os.path.join(input_folder, filename)
        manager2.add(
            data=file_path,
            id = str(os.path.splitext(filename)[0])
        )
        print(f"Processed: {filename}")

# Add data to manager (with sensors)
filename = 'ni_3D.npz'
file_path = os.path.join(input_folder, filename)
ni_3D = np.load(file_path)
ni_3D = ni_3D[ni_3D.files[0]]

manager2.add(
    data=ni_3D,
    stationary_sensors=[(29, 201), (76, 26), (11, 247)],
    id = str(os.path.splitext(filename)[0])
)


filename Jez_3D.npz
Processed: Jez_3D.npz
filename ne_3D.npz
Processed: ne_3D.npz
filename Jey_3D.npz
Processed: Jey_3D.npz
filename EfZ_3D.npz
Processed: EfZ_3D.npz
filename Vdiz_3D.npz
Processed: Vdiz_3D.npz
filename Tex_3D.npz
Processed: Tex_3D.npz
filename EfX_3D.npz
Processed: EfX_3D.npz
filename Jex_3D.npz
Processed: Jex_3D.npz
filename ni_3D.npz
filename Vdiy_3D.npz
Processed: Vdiy_3D.npz
filename phi_3D.npz
Processed: phi_3D.npz
filename Ti_3D.npz
Processed: Ti_3D.npz
filename Vdix_3D.npz
Processed: Vdix_3D.npz
filename Tez_3D.npz
Processed: Tez_3D.npz


In [12]:
manager2.input_summary

,field id,sensor id,sensor type,location/trajectory
0,ni_3D,ni_3D-0,stationary (user selected),"(29, 201)"
1,ni_3D,ni_3D-1,stationary (user selected),"(76, 26)"
2,ni_3D,ni_3D-2,stationary (user selected),"(11, 247)"


In [13]:
manager2.sensor_measurements

,time,ni_3D-0,ni_3D-1,ni_3D-2
0,0,5.263100e+16,5.625600e+16,5.009700e+16
1,1,5.315000e+16,5.802500e+16,5.040600e+16
2,2,5.405600e+16,5.980300e+16,4.995000e+16
3,3,5.479300e+16,5.928200e+16,4.755900e+16
4,4,5.432100e+16,5.721300e+16,4.576400e+16
...,...,...,...,...
1995,1995,7.625200e+16,1.036000e+17,4.160500e+16
1996,1996,7.924100e+16,1.044900e+17,4.053400e+16
1997,1997,8.920600e+16,1.165200e+17,4.258400e+16
1998,1998,8.696500e+16,1.371100e+17,4.459300e+16


In [21]:
# Get train/val/test datasets
train_set_2, val_set_2, test_set_2 = manager2.preprocess()

# Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set_2.reconstructor_dataset.X.shape)
print('train Y:', train_set_2.reconstructor_dataset.Y.shape)
print('val X:', val_set_2.reconstructor_dataset.X.shape)
print('val Y:', val_set_2.reconstructor_dataset.Y.shape)
print('test X:', test_set_2.reconstructor_dataset.X.shape)
print('test Y:', test_set_2.reconstructor_dataset.Y.shape)

print ('Predictor Data')
print('train X:', train_set_2.predictor_dataset.X.shape)
print('train Y:', train_set_2.predictor_dataset.Y.shape)
print('val X:', val_set_2.predictor_dataset.X.shape)
print('val Y:', val_set_2.predictor_dataset.Y.shape)
print('test X:', test_set_2.predictor_dataset.X.shape)
print('test Y:', test_set_2.predictor_dataset.Y.shape)

print ('Sensor Forecaster Data')
print('train X:', train_set_2.sensor_forecaster_dataset.X.shape)
print('train Y:', train_set_2.sensor_forecaster_dataset.Y.shape)
print('val X:', val_set_2.sensor_forecaster_dataset.X.shape)
print('val Y:', val_set_2.sensor_forecaster_dataset.Y.shape)
print('test X:', test_set_2.sensor_forecaster_dataset.X.shape)
print('test Y:', test_set_2.sensor_forecaster_dataset.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([1200, 21, 3])
train Y: torch.Size([1200, 2800])
val X: torch.Size([400, 21, 3])
val Y: torch.Size([400, 2800])
test X: torch.Size([400, 21, 3])
test Y: torch.Size([400, 2800])
Predictor Data
train X: torch.Size([1200, 21, 3])
train Y: torch.Size([1200, 2800])
val X: torch.Size([400, 21, 3])
val Y: torch.Size([400, 2800])
test X: torch.Size([400, 21, 3])
test Y: torch.Size([400, 2800])
Sensor Forecaster Data
train X: torch.Size([1200, 21, 3])
train Y: torch.Size([1200, 3])
val X: torch.Size([400, 21, 3])
val Y: torch.Size([400, 3])
test X: torch.Size([400, 21, 3])
test Y: torch.Size([400, 3])


In [22]:
# initialize SHRED
shred2 = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred2.fit(train_set_2, val_set_2, num_epochs=num_epochs, patience=100, verbose=True)


Fitting Reconstructor...


Epoch 1/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

Epoch 2/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

Epoch 3/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

Epoch 4/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

...

Epoch 162/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

Epoch 163/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

Epoch 164/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

Epoch 165/2000:   0%|          | 0/19 [00:00<?, ?batch/s]

Early stopping triggered: patience threshold reached.


In [23]:
evaluate(shred = shred2, test_dataset=test_set_2, data_manager=manager2, uncompress=True, unscale = True)

,Jez_3D,ne_3D,Jey_3D,EfZ_3D,Vdiz_3D,Tex_3D,EfX_3D,Jex_3D,Vdiy_3D,phi_3D,Ti_3D,Vdix_3D,Tez_3D,ni_3D
reconstruction,0.063210,0.064224,0.450594,0.607206,0.526880,0.059954,0.078711,0.425413,0.048367,0.053280,0.084430,0.057341,0.047674,0.089045
prediction,0.072428,0.088221,0.893335,1.048476,0.961397,0.092690,0.102812,0.952601,0.038138,0.066114,0.104226,0.062241,0.060500,0.132633
forecast,0.072137,0.087415,0.867182,1.021601,0.938991,0.093323,0.099672,0.922042,0.035804,0.063872,0.101452,0.063137,0.059689,0.130116
sensor_forecast,0.112718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
test_set_2.reconstructor_dataset.X = test_set_2.predictor_dataset.X
test_set_2.reconstructor_dataset.Y = test_set_2.predictor_dataset.Y

In [25]:
# PREDICTION (NOT RECONSTRUCTION) WITH "SINGLE_MODEL" Version
evaluate(shred = shred, test_dataset=test_set_2, data_manager=manager, uncompress=True, unscale = True)

,Jez_3D,ne_3D,Jey_3D,EfZ_3D,Vdiz_3D,Tex_3D,EfX_3D,Jex_3D,Vdiy_3D,phi_3D,Ti_3D,Vdix_3D,Tez_3D,ni_3D
reconstruction,0.100938,0.105597,0.983026,1.106945,0.974734,0.102241,0.132418,0.976853,0.044806,0.083795,0.124835,0.077223,0.066267,0.156005
